In [1]:
import model
import fastText
from keras.callbacks import ModelCheckpoint
from importlib import reload

Using TensorFlow backend.


In [2]:
from keras.backend.tensorflow_backend import set_session
import tensorflow as tf

%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=0

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
set_session(tf.Session(config=config))

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=0


In [8]:
reload(model)

<module 'model' from '/srv/home/gwiedemann/pss-lre/model.py'>

In [4]:
data_train = model.read_csv_data("data/archive20k/text/dataset.train")
data_test = model.read_csv_data("data/archive20k/text/dataset.test")

In [5]:
# print(model.simple_tokenizer(data_train[12][2]))

In [6]:
if 'ft' not in locals():
    ft = fastText.load_model("../embeddings/wiki.de.bin")
    model.ft = ft

In [21]:
# model.vocab = {}
# model.vocab["PADDING_TOKEN"] = 0
# model.vocab["UNKNOWN_TOKEN"] = 1
# for d in data_train:
#     sentence = model.simple_tokenizer(d[2])
#     for w in sentence:
#         model.vocab[w] = len(model.vocab)
# # print(vocab)
# model.nb_words = len(model.vocab)
# print(model.nb_words)

# Experiment: Simple CNN model (single page)
* optimize for Kappa
* 10 repeats

In [9]:
n_repeats = 10
n_epochs = 10
exp_history = []
optimize_for = 'kappa'
for i in range(n_repeats):
    print("Repeat " + str(i+1) + " of " + str(n_repeats))
    print("-------------------------")
    model_singlepage = model.compile_model_singlepage()
    model_file = "exp1_single-page_repeat-%02d.hdf5" % (i,)
    print(model_file)
    checkpoint = model.ValidationCheckpoint(model_file, data_test, metric=optimize_for)
    model_singlepage.fit_generator(generator=model.TextFeatureGenerator(data_train),
                    callbacks = [checkpoint],
                    epochs = n_epochs)
    exp_history.append(checkpoint.max_metrics)

avg_result = sum([m['kappa'] for m in exp_history]) / n_repeats
print("-------------------------")
print(avg_result)

Repeat 1 of 10
-------------------------
exp1_single-page_repeat-00.hdf5
Epoch 1/10
543/543 [==============================] - 233s 428ms/step - loss: 0.4549 - acc: 0.8199

kappa improvement: 0.46286148677867656 (before: -inf), saving to exp1_single-page_repeat-00.hdf5
Epoch 2/10
543/543 [==============================] - 233s 429ms/step - loss: 0.2862 - acc: 0.8821

kappa improvement: 0.5257085270920605 (before: 0.46286148677867656), saving to exp1_single-page_repeat-00.hdf5
Epoch 3/10
543/543 [==============================] - 237s 436ms/step - loss: 0.2434 - acc: 0.8992

kappa improvement: 0.5814593827118759 (before: 0.5257085270920605), saving to exp1_single-page_repeat-00.hdf5
Epoch 4/10
543/543 [==============================] - 232s 427ms/step - loss: 0.2195 - acc: 0.9087

kappa improvement: 0.5973239059585371 (before: 0.5814593827118759), saving to exp1_single-page_repeat-00.hdf5
Epoch 5/10
543/543 [==============================] - 235s 433ms/step - loss: 0.1935 - acc: 0.9170


In [14]:
# average: 0.5828771586417484, 0.6197080667414943
for i, r in enumerate(exp_history):
    model_file = "exp1_single-page_repeat-%02d.hdf5" % (i,)
    print(str(i) + ' ' + str(r['kappa']) + ' ' + str(r['accuracy']) + ' ' + str(r['f1_macro']) + ' ' + model_file)

0 0.6068494755504181 0.9030421982335624 0.8034246763641316 exp1_single-page_repeat-00.hdf5
1 0.6143211518297829 0.912855740922473 0.8067899969252845 exp1_single-page_repeat-01.hdf5
2 0.6154014731843915 0.9148184494602551 0.8071451945337149 exp1_single-page_repeat-02.hdf5
3 0.625308609549824 0.9038272816486752 0.8125853914871257 exp1_single-page_repeat-03.hdf5
4 0.6201653323980538 0.9103042198233562 0.8099907778715595 exp1_single-page_repeat-04.hdf5
5 0.6277742788764329 0.9089303238469087 0.8138833885051218 exp1_single-page_repeat-05.hdf5
6 0.622023004198657 0.9114818449460255 0.8108826982950066 exp1_single-page_repeat-06.hdf5
7 0.632113767445956 0.9114818449460255 0.816027061378489 exp1_single-page_repeat-07.hdf5
8 0.6132797675408165 0.9028459273797841 0.8066256362333042 exp1_single-page_repeat-08.hdf5
9 0.6198438068406102 0.9053974484789009 0.8099190198165531 exp1_single-page_repeat-09.hdf5


# Experiment: Predecessor page

In [8]:
import os, warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
warnings.filterwarnings(action='once')

In [ ]:
# [features_page, features_dot]
n_repeats = 10
n_epochs = 10
exp1b_history = []
optimize_for = 'kappa'
for i in range(n_repeats):
    print("Repeat " + str(i+1) + " of " + str(n_repeats))
    print("-------------------------")
    model_prevpage = model.compile_model_prevpage()
    model_file = "exp1_prev-page_repeat-%02d.hdf5" % (i,)
    print(model_file)
    checkpoint = model.ValidationCheckpoint(model_file, data_test, prev_page_generator = True, metric=optimize_for)
    model_prevpage.fit_generator(generator=model.TextFeatureGenerator2(data_train),
                    callbacks = [checkpoint],
                    epochs = n_epochs)
    exp1b_history.append(checkpoint.max_metrics)

avg_result = sum([m['kappa'] for m in exp1b_history]) / n_repeats
print("-------------------------")
print(avg_result)

Repeat 1 of 10
-------------------------
exp1_prev-page_repeat-00.hdf5
Epoch 1/10
543/543 [==============================] - 395s 727ms/step - loss: 0.5213 - acc: 0.8184

kappa improvement: 0.48614444163816384 (before: -inf), saving to exp1_prev-page_repeat-00.hdf5
Epoch 2/10
543/543 [==============================] - 394s 726ms/step - loss: 0.2680 - acc: 0.8869

kappa improvement: 0.5670538037913879 (before: 0.48614444163816384), saving to exp1_prev-page_repeat-00.hdf5
Epoch 3/10
543/543 [==============================] - 393s 724ms/step - loss: 0.2189 - acc: 0.9079

kappa improvement: 0.5944952472379827 (before: 0.5670538037913879), saving to exp1_prev-page_repeat-00.hdf5
Epoch 4/10
543/543 [==============================] - 392s 722ms/step - loss: 0.1914 - acc: 0.9186
Epoch 5/10
543/543 [==============================] - 390s 718ms/step - loss: 0.1780 - acc: 0.9277

kappa improvement: 0.6192371615348466 (before: 0.5944952472379827), saving to exp1_prev-page_repeat-00.hdf5
Epoch 6/10

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



543/543 [==============================] - 397s 731ms/step - loss: 0.1096 - acc: 0.9567
Repeat 6 of 10
-------------------------
exp1_prev-page_repeat-05.hdf5
Epoch 1/10
543/543 [==============================] - 428s 788ms/step - loss: 0.4929 - acc: 0.8131

kappa improvement: 0.15975931359803885 (before: -inf), saving to exp1_prev-page_repeat-05.hdf5
Epoch 2/10
543/543 [==============================] - 442s 814ms/step - loss: 0.2698 - acc: 0.8861

kappa improvement: 0.5315110153042515 (before: 0.15975931359803885), saving to exp1_prev-page_repeat-05.hdf5
Epoch 3/10
543/543 [==============================] - 446s 822ms/step - loss: 0.2201 - acc: 0.9054

kappa improvement: 0.6276786561639935 (before: 0.5315110153042515), saving to exp1_prev-page_repeat-05.hdf5
Epoch 4/10
543/543 [==============================] - 445s 820ms/step - loss: 0.1935 - acc: 0.9167
Epoch 5/10
543/543 [==============================] - 445s 819ms/step - loss: 0.1745 - acc: 0.9247
Epoch 6/10
237/543 [===========

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



543/543 [==============================] - 436s 804ms/step - loss: 0.2167 - acc: 0.9091

kappa improvement: 0.6239868494839536 (before: 0.560312405788363), saving to exp1_prev-page_repeat-06.hdf5
Epoch 4/10
543/543 [==============================] - 435s 802ms/step - loss: 0.1889 - acc: 0.9174
Epoch 5/10
543/543 [==============================] - 436s 804ms/step - loss: 0.1741 - acc: 0.9256

kappa improvement: 0.6265342882707583 (before: 0.6239868494839536), saving to exp1_prev-page_repeat-06.hdf5
Epoch 6/10
543/543 [==============================] - 436s 802ms/step - loss: 0.1562 - acc: 0.9346

kappa improvement: 0.627609702809225 (before: 0.6265342882707583), saving to exp1_prev-page_repeat-06.hdf5
Epoch 7/10
543/543 [==============================] - 439s 808ms/step - loss: 0.1422 - acc: 0.9407
Epoch 8/10
543/543 [==============================] - 435s 800ms/step - loss: 0.1324 - acc: 0.9451

kappa improvement: 0.6404606107261996 (before: 0.627609702809225), saving to exp1_prev-page

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



543/543 [==============================] - 436s 803ms/step - loss: 0.1178 - acc: 0.9511
Epoch 10/10
543/543 [==============================] - 439s 808ms/step - loss: 0.1141 - acc: 0.9536
Repeat 8 of 10
-------------------------
exp1_prev-page_repeat-07.hdf5
Epoch 1/10
543/543 [==============================] - 445s 819ms/step - loss: 0.4986 - acc: 0.8135

kappa improvement: 0.2443045431967562 (before: -inf), saving to exp1_prev-page_repeat-07.hdf5
Epoch 2/10
543/543 [==============================] - 441s 813ms/step - loss: 0.2646 - acc: 0.8870

kappa improvement: 0.6243437360961037 (before: 0.2443045431967562), saving to exp1_prev-page_repeat-07.hdf5
Epoch 3/10
543/543 [==============================] - 450s 829ms/step - loss: 0.2215 - acc: 0.9052
Epoch 4/10
503/543 [==========================>...] - ETA: 33s - loss: 0.1945 - acc: 0.9192

In [11]:
for i, r in enumerate(exp1b_history):
    model_file = "exp1_prev-page_repeat-%02d.hdf5" % (i,)
    print(str(i) + ' ' + str(r['kappa']) + ' ' + str(r['accuracy']) + ' ' + model_file)

0 0.6192371615348466 0.9040235525024534 exp1_prev-page_repeat-00.hdf5
1 0.6467880796033505 0.9154072620215898 exp1_prev-page_repeat-01.hdf5
2 0.6291333109443216 0.9085377821393523 exp1_prev-page_repeat-02.hdf5
3 0.6323662560145851 0.9079489695780176 exp1_prev-page_repeat-03.hdf5
4 0.6066653746144032 0.9061825318940138 exp1_prev-page_repeat-04.hdf5
5 0.6276786561639935 0.9093228655544652 exp1_prev-page_repeat-05.hdf5
6 0.6404606107261996 0.9136408243375859 exp1_prev-page_repeat-06.hdf5
7 0.6258252471379041 0.9050049067713445 exp1_prev-page_repeat-07.hdf5
8 0.6284768130895626 0.9122669283611383 exp1_prev-page_repeat-08.hdf5
9 0.6314968169247022 0.9103042198233562 exp1_prev-page_repeat-09.hdf5


In [ ]:
# load best model
model.load_weights("exp1_prevapge_model.hdf5")
y_predict = model.predict_generator(TextFeatureGenerator2(data_test, batch_size=256)).argmax(axis=-1)
y_true = [label2Idx[x[1]] for x in data_test]
print("Accuracy: " + str(sklm.accuracy_score(y_true, y_predict)))
print("Kappa: " + str(sklm.cohen_kappa_score(y_true, y_predict)))